In [9]:
import sys  
sys.path.insert(0, '../AuxiliarCodes/')

In [10]:
import gurobipy   as     gp
from   gurobipy   import GRB
from drawpointFunction  import drawpointFunction
from globalFunctions import getNumberOfBlocksInADimension
import pandas as pd
from functools import reduce

In [11]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'


In [12]:
undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook

In [13]:
class UndergroundModel:
    def __init__(self, database, numberOfPeriods):
        self.database = database
        self.numberOfPeriods = numberOfPeriods
        self.DP_init = 0       #### Tipo de extracción
        self.desc = 0.1
        self.colHeight = 300
        self.minColHeight = 0.60
        self.pos_x = 430     
        self.pos_y = 550     
        self.pos_z = 780
        self.pos_x_f = 730     
        self.pos_y_f = 910     
        self.p_t = 3791.912
        self.epsilon = 1
        self.orientationToExtractTheDrawpoints = 0

    def execute(self):
        self.getUndergroundVariablesFromCSV()
        self.getUndergroundInfo()
        self.setUndergroundParameters()
        self.setUndergroundMineLimits()
        self.setUndergroundVariables()
        self.setModelandGetResults()
        return self.objValue, self.variableValues, self.runtime, self.gap
    
    def getUndergroundVariablesFromCSV(self):
        self.undergroundBlocksLength = self.database['X'].to_dict()             
        self.undergroundBlocksWidth  = self.database['Y'].to_dict()             
        self.undergroundBlocksHeight = self.database['Z'].to_dict()             
        self.undergroundBlockTonnage = self.database['Ton'].to_dict()              
        self.undergroundBlockMineral  = self.database['Mineral'].to_dict()          
        self.undergroundBlockRecovery  = self.database['Recuperación'].to_dict()     
        self.undergroundCopperLaw  = self.database['%Cu'].to_dict()
        self.undergroundExtractionFixedCosts = self.database['CPlanta CA'].to_dict()
        self.undergroundVariableExtractionCosts = self.database['CMina CA'].to_dict()
        self.undergroundCP_S = self.database['CPlanta S'].to_dict()
        self.undergroundCM_S = self.database['CMINA S'].to_dict() 
    
    def getUndergroundInfo(self):
        self.undergroundBlocks = [i for i in range(len(self.undergroundBlocksLength.values()))]

    def setUndergroundParameters(self):
        #Underground Parameters
        self.t_S   = {period : period + 1 for period in range(self.numberOfPeriods)}
        self.MU_mt = {period : 25806600.0/3  for period in range(self.numberOfPeriods)} #Tonleage es mina
        self.ML_mt = {period : 0.0  for period in range(self.numberOfPeriods)}
        self.MU_pt = {period : 17777880.0/3  for period in range(self.numberOfPeriods)}#Mineral es planta
        self.ML_pt = {period : 0.0 for period in range(self.numberOfPeriods)}
        self.qU_dt = {period : 1 for period in range(self.numberOfPeriods)}
        self.qL_dt = {period : 0.0001 for period in range(self.numberOfPeriods)}
        self.A_d   = {period : 2 for period in range(self.numberOfPeriods)}
        self.NU_nt = {period : 59 for period in range(self.numberOfPeriods)} 
        self.NL_nt = {period : 0 for period in range(self.numberOfPeriods)}
        self.N_t   = {period : 57* (1 + period) for period in range(self.numberOfPeriods)}
        self.RL_dt = {period : 0.3 for period in range(self.numberOfPeriods)}
        self.RU_dt = {period : 0.7 for period in range(self.numberOfPeriods)}

    def setUndergroundVariables(self):
        self.drawpoint, self.G_d, self.Q_d,self.q_d, self.C_pdt, self.C_mdt, self.predecessor, self.x_draw,self.y_draw, self.z_draw = drawpointFunction(
                        self.pos_x, self.pos_y, self.pos_z, self.colHeight, self.DP_init, self.undergroundBlocksLengthLimits, self.undergroundBlocksWidthLimits, self.undergroundBlocksHeightLimits, self.undergroundBlockTonnage, self.undergroundCP_S, self.undergroundCM_S, self.undergroundBlockMineral,
                        self.undergroundCopperLaw, self.pos_x_f, self.pos_y_f,self.orientationToExtractTheDrawpoints)
        #G_d = waste and minneral tonnelage 
        #Q_d = minneral tonnelage
        self.predecessorDict = {}
        self.predecessorDict[0] = []
        self.predecessorDict[1] = [0]
        for i in range(1,len(self.predecessor)):
            if self.predecessor[i][0] not in self.predecessorDict.keys():
                self.predecessorDict[self.predecessor[i][0]] = []
            self.predecessorDict[self.predecessor[i][0]].append(self.predecessor[i][1])
    
    def setUndergroundMineLimits(self):
        self.undergroundBlocksLengthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksLength)
        self.undergroundBlocksWidthLimits = getNumberOfBlocksInADimension(self.undergroundBlocksWidth)
        self.undergroundBlocksHeightLimits = getNumberOfBlocksInADimension(self.undergroundBlocksHeight)

    def setModelandGetResults(self):
        self.objValue, self.variableValues, self.runtime, self.gap = self.setUndergroundModel()#v_p, theta_opt, w_opt)

    def setUndergroundModel(self):#,v_p, theta_opt, w_opt):
        self.undergroundModel = gp.Model(name = 'Modelo Integrado')
        self.undergroundModel.Params.TimeLimit = 3600

        #14. Naturaleza de las variables
        x_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="x")
        y_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.CONTINUOUS, name="y")
        z_dt = self.undergroundModel.addVars(self.drawpoint, self.t_S, vtype=GRB.BINARY, name="z")

        #1. Restricción sobre la cantidad de tonelaje máxima y mínima a extraer en cada periodo.
        Ton_Up = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti]*self.G_d[d] for d in self.drawpoint) <= self.MU_mt[ti] for ti in self.t_S),
                                         "Min_max")
        
        Ton_low = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti] * self.G_d[d] for d in self.drawpoint) >= self.ML_mt[ti] for ti in self.t_S),
                                            "Min_min")
        #2. Restricción sobre la cantidad de material máxima y mínima a procesar en cada periodo.
        Mat_Up = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) <= self.MU_pt[ti] for ti in self.t_S),
                                            "Mat_max")

        Mat_low = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti] * self.Q_d[d] for d in self.drawpoint) >= self.ML_pt[ti] for ti in self.t_S)
                                            , "Mat_min")
        #3. Rango de leyes máximas y mínimas a procesar
        GQC_low = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * y_dt[d, ti] for d in self.drawpoint) >=
                                self.qL_dt[ti] * gp.quicksum(self.G_d[d] * y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_low")
        
        GQC_Up = self.undergroundModel.addConstrs((gp.quicksum(self.Q_d[d] * self.q_d[d] * y_dt[d, ti] for d in self.drawpoint) <=
                                self.qU_dt[ti] * gp.quicksum(self.G_d[d] * y_dt[d, ti] for d in self.drawpoint) for ti in self.t_S), "GQC_Up")

        #4. Todos los puntos de extracción deben ser iniciados en el largo de la extracción
        Drawp_init = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint), "Drawp_init")
        Drawp_init_2 = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for ti in self.t_S) >= 0.1 for d in self.drawpoint), "Drawp_init_2")

        #5. Los puntos de extracción deben ser activados al menos en el mismo periodo para que se inicie la extracción 
        Drawpextract_61 = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, tau] for tau in range(ti+1)) >= z_dt[d, ti]  
                                            for d in self.drawpoint for ti in self.t_S), "Drawpextract_61")


        #6. Existe una cantidad máxima y mínima de drawpoints a abrir en cada periodo.
        Drawpextract_64_1 = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) <= self.NU_nt[ti] for ti 
                                                        in self.t_S)
                                                        ,"Drawpextract_64_1")
        Drawpextract_64_2 = self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) >= self.NL_nt[ti] for ti 
                                                        in self.t_S)
                                                        , "Drawpextract_64_2")

        #7. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        Drawpextract_65 = self.undergroundModel.addConstrs((gp.quicksum(z_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")


        #8. Si iniciamos la extracción de un drawpoint esta debe durar por su duraci ́on determinada.
        ## Un drawpoint solamente puede ser extraido por un preiodo pre determinado (A_d)
        Drawpextract_62 = self.undergroundModel.addConstrs((gp.quicksum(z_dt[d, ti] for ti in self.t_S)  <= self.A_d[ti]  for d in self.drawpoint
                                                    for ti in self.t_S), "Drawp_62")

        ## Una vez se inicia extrayendo de un drawpoint, se continua extrayendo sin interrupción
        Drawpextract_63 = self.undergroundModel.addConstrs((self.A_d[ti] *(z_dt[d, ti] - z_dt[d, ti+1]) 
                                            - gp.quicksum(z_dt[d, tau] for tau in range(ti+1)) <= 0 
                                            for d in self.drawpoint for ti in range(0,max(self.t_S))), "Drawpextract_63")

        #9. Relación de variables, el porcentaje a extraer es 0 si no se extra un drawpoint.
        Drawpextract_66 = self.undergroundModel.addConstrs((y_dt[d, ti] <= z_dt[d, ti] for d in self.drawpoint for ti in self.t_S),
                                                    "Drawpextract_66")

        #10. Existe una tasa m ́ınima de extracci ́on para cada drawpoint a extraer.
        Drawpextract_67_1 = self.undergroundModel.addConstrs((self.RL_dt[ti] * z_dt[d, ti]  <=  y_dt[d, ti] for d in self.drawpoint
                                                        for ti in self.t_S), "Drawpextract_67_1")

        #11. La altura a extraer debe ser mayor a una cantidad m ́ınima.
        rest_11 = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d,ti] for ti in self.t_S)>= self.minColHeight for d in self.drawpoint))

        #12. No podemos extraer más del 100 % de un drawpoint.
        Reserver_cnst = self.undergroundModel.addConstrs((gp.quicksum(y_dt[d, ti] for ti in self.t_S) <= 1 for d in self.drawpoint),
                                                    "Reserver_cnst")

        #13. Si se activa un drawpoint, se extrae en ese periodo
        rest_13 = self.undergroundModel.addConstrs(x_dt[d,ti] <= z_dt[d, ti] for d in self.drawpoint for ti in self.t_S)

        #14. Naturaleza de variables.

        #15. Existe una m ́axima cantidad de drawpoints a extraer por periodo.
        rest_15= self.undergroundModel.addConstrs((gp.quicksum(x_dt[d, ti] for d in self.drawpoint) <= self.N_t[ti] for ti in self.t_S)
                                                    , "Drawpextract_65")
        
        #16. Restricción sobre el inicio de la extracci ́on de los drawpoints.
        alternative = self.undergroundModel.addConstrs(gp.quicksum(x_dt[a,s] for s in range(0,ti+1)) >= x_dt[d, ti] for d in self.drawpoint for ti in self.t_S for a in self.predecessorDict[d])


        #Función objetivo
        undergroundObjectiveFunction = gp.quicksum(y_dt[d, ti]*((((self.p_t * self.q_d[d] -self.C_pdt[d] ) * self.Q_d[d])-(self.C_mdt[d]*self.G_d[d]))/
                                        ((1+self.desc)**(self.t_S[ti]))) for ti in self.t_S for d in self.drawpoint)
        
       
        self.undergroundModel.setObjective(undergroundObjectiveFunction, GRB.MAXIMIZE)
        self.undergroundModel.Params.MIPGap = 0.01
        
        self.undergroundModel.optimize()
        lista_variable_Integrado = (self.undergroundModel.getAttr(GRB.Attr.X, self.undergroundModel.getVars()))
        solucion = self.undergroundModel.objVal
        runtime = self.undergroundModel.Runtime
        gap_f = self.undergroundModel.MIPGap

        return solucion, lista_variable_Integrado, runtime, gap_f

In [14]:
numberOfPeriods = 10

In [15]:
undergroundModel = UndergroundModel(undergroundMineDataframe,numberOfPeriods)

In [16]:
undergroundObjValue, undergroundVariableValues, undergroundRuntime, undergroundGap = undergroundModel.execute()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-02
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 13230 rows, 5400 columns and 87055 nonzeros
Model fingerprint: 0xb33321ca
Variable types: 1800 continuous, 3600 integer (3600 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+05]
  Objective range  [4e+05, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 3e+07]
Presolve removed 5265 rows and 959 columns
Presolve time: 0.32s
Presolved: 7965 rows, 4441 columns, 35010 nonzeros
Variable types: 1606 continuous, 2835 integer (2835 binary)
Found heuristic solution: objective 1.932350e+09

Root relaxation: objective 2.070211e+09, 4440 iterations, 0.05 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     W

Best objective 2.059010099765e+09, best bound 2.070596030476e+09, gap 0.5627%, 2.3 secs


Best objective 2.068401014398e+09, best bound 2.068639475042e+09, gap 0.0115%

Best objective 2.068400680349e+09, best bound 2.068639971131e+09, gap 0.0116%







In [17]:
undergroundModel.predecessorDict

{0: [],
 1: [0],
 15: [1],
 2: [15],
 16: [2],
 30: [16],
 3: [30],
 17: [3],
 31: [17],
 45: [31],
 4: [45],
 18: [4],
 32: [18],
 46: [32],
 60: [46],
 5: [60],
 19: [5],
 33: [19],
 47: [33],
 61: [47],
 75: [61],
 6: [75],
 20: [6],
 34: [20],
 48: [34],
 62: [48],
 76: [62],
 90: [76],
 7: [90],
 21: [7],
 35: [21],
 49: [35],
 63: [49],
 77: [63],
 91: [77],
 105: [91],
 8: [105],
 22: [8],
 36: [22],
 50: [36],
 64: [50],
 78: [64],
 92: [78],
 106: [92],
 120: [106],
 9: [120],
 23: [9],
 37: [23],
 51: [37],
 65: [51],
 79: [65],
 93: [79],
 107: [93],
 121: [107],
 135: [121],
 10: [135],
 24: [10],
 38: [24],
 52: [38],
 66: [52],
 80: [66],
 94: [80],
 108: [94],
 122: [108],
 136: [122],
 150: [136],
 11: [150],
 25: [11],
 39: [25],
 53: [39],
 67: [53],
 81: [67],
 95: [81],
 109: [95],
 123: [109],
 137: [123],
 151: [137],
 165: [151],
 12: [165],
 26: [12],
 40: [26],
 54: [40],
 68: [54],
 82: [68],
 96: [82],
 110: [96],
 124: [110],
 138: [124],
 152: [138],
 166: 

In [18]:
undergroundModel.predecessorDict

{0: [],
 1: [0],
 15: [1],
 2: [15],
 16: [2],
 30: [16],
 3: [30],
 17: [3],
 31: [17],
 45: [31],
 4: [45],
 18: [4],
 32: [18],
 46: [32],
 60: [46],
 5: [60],
 19: [5],
 33: [19],
 47: [33],
 61: [47],
 75: [61],
 6: [75],
 20: [6],
 34: [20],
 48: [34],
 62: [48],
 76: [62],
 90: [76],
 7: [90],
 21: [7],
 35: [21],
 49: [35],
 63: [49],
 77: [63],
 91: [77],
 105: [91],
 8: [105],
 22: [8],
 36: [22],
 50: [36],
 64: [50],
 78: [64],
 92: [78],
 106: [92],
 120: [106],
 9: [120],
 23: [9],
 37: [23],
 51: [37],
 65: [51],
 79: [65],
 93: [79],
 107: [93],
 121: [107],
 135: [121],
 10: [135],
 24: [10],
 38: [24],
 52: [38],
 66: [52],
 80: [66],
 94: [80],
 108: [94],
 122: [108],
 136: [122],
 150: [136],
 11: [150],
 25: [11],
 39: [25],
 53: [39],
 67: [53],
 81: [67],
 95: [81],
 109: [95],
 123: [109],
 137: [123],
 151: [137],
 165: [151],
 12: [165],
 26: [12],
 40: [26],
 54: [40],
 68: [54],
 82: [68],
 96: [82],
 110: [96],
 124: [110],
 138: [124],
 152: [138],
 166: 

In [19]:
undergroundModel.predecessorDict = {}
undergroundModel.predecessorDict[0] = []
undergroundModel.predecessorDict[1] = [0]
for i in range(1,len(undergroundModel.predecessor)):
    if undergroundModel.predecessor[i][0] not in undergroundModel.predecessorDict.keys():
        undergroundModel.predecessorDict[undergroundModel.predecessor[i][0]] = []
    undergroundModel.predecessorDict[undergroundModel.predecessor[i][0]].append(undergroundModel.predecessor[i][1])


In [20]:
self.predecessorDict = {}
self.predecessorDict[0] = []
self.predecessorDict[1] = [0]
for i in range(1,len(self.predecessor)):
    self.predecessorDict[self.predecessor[i][0]] =  [*reduce(lambda x, y: set(x) | set(y), self.predecessor[:i])]

NameError: name 'self' is not defined